<p style='margin-top: 15px; text-align: center; font-size: 22px;'><b>Corrigindo Casos de Subnotificação da COVID-19 em Fortaleza/CE</b></p>


<div style="margin-top: 30px;">
    <p style="font-size: 20px; text-align: center;"><b>João Manoel Lins</b></p>
</div>

In [1]:
import os
import glob
import zipfile, requests, io

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
# Apagando os dados do coronavirus antigos.
for f in glob.glob('casos_coronavirus*.csv'):
    os.remove(f)

# Url download do IntegraSUS
FILE_URL = 'http://download-integrasus.saude.ce.gov.br/download'

# Baixando os dados atualizados
r = requests.get(FILE_URL)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(os.getcwd())

In [ ]:
CASOS_NAME = 'casos_coronavirus*.csv'
CASOS_PATH = os.path.join(os.getcwd(), CASOS_NAME)

FAIXA_ETARIA_NAME = 'FAIXA_ETARIA_FORTALEZA_2019.csv'
FAIXA_ETARIA_PATH = os.path.join(os.getcwd(), FAIXA_ETARIA_NAME)

FAIXA_ETARIA_COREIA_NAME = 'FAIXA_ETARIA_COREIA_SUL_2020.csv'
FAIXA_ETARIA_COREIA_PATH = os.path.join(os.getcwd(), FAIXA_ETARIA_COREIA_NAME)

# Importando os arquivos
csvs = (pd.read_csv(f, sep=',', low_memory=False) for f in glob.glob(CASOS_PATH))

# concatenando todos os arquivos em um único dataframe
casos = pd.concat(csvs, ignore_index=True)

faixa_etaria = pd.read_csv(FAIXA_ETARIA_PATH, sep=',')
faixa_etaria_coreia = pd.read_csv(FAIXA_ETARIA_COREIA_PATH, sep=',')

### Limpeza no DataFrame de Casos/Óbitos

In [ ]:
# Removendo coluna
casos.drop(['Unnamed: 0'], axis=1, inplace=True)

# Transformando as colunas em maiúscula
casos.columns = list(map(lambda x: x.upper(), casos.columns))

In [ ]:
# Casos de Fortaleza
casos = casos[casos['MUNICIPIOPACIENTE'] == 'FORTALEZA']

# Somente casos positivos
casos_confirmados = casos[casos['RESULTADOFINALEXAME'] == 'Positivo']

# Removendo casos sem data de notificacao
casos_confirmados = casos_confirmados[~casos_confirmados['DATANOTIFICACAO'].isna()]

# Todos obitos confirmados possuem data
obitos = casos[casos['OBITOCONFIRMADO'] == True]

### Agrupando por semana

#### Agrupando por Semanas

In [ ]:
# Convertendo a coluna para datetime
casos_confirmados['DATANOTIFICACAO'] = pd.to_datetime(casos_confirmados['DATANOTIFICACAO'])
obitos['DATAOBITO'] = pd.to_datetime(obitos['DATAOBITO'])

In [ ]:
# Criando a coluna com a semana do ano
casos_confirmados['SEMANA'] = casos_confirmados['DATANOTIFICACAO'].dt.strftime('%W')
obitos['SEMANA'] = obitos['DATAOBITO'].dt.strftime('%W')

In [ ]:
# Quantidade de casos/óbitos por semana do ano
casos_semana_index = {week: index[0] for week, index in sorted(casos_confirmados.groupby('SEMANA').groups.items(), reverse=False)}
casos_confirmados_semana = casos_confirmados.groupby(casos_confirmados['DATANOTIFICACAO'].dt.strftime('%W'))['CODIGOPACIENTE'].count()

obitos_semana_index = {week: index[0] for week, index in sorted(obitos.groupby('SEMANA').groups.items(), reverse=False)}
obitos_semana = casos_confirmados.groupby(obitos['DATAOBITO'].dt.strftime('%W'))['DATAOBITO'].count()

---

## Gráficos

### Casos/Óbitos

In [ ]:
values = casos_confirmados_semana.keys() if len(casos_confirmados_semana.keys()) > len(obitos_semana.keys()) else obitos_semana.keys()
for v in values:
    if v not in casos_confirmados_semana.keys():
        casos_confirmados_semana = casos_confirmados_semana.append(pd.Series({v: 0}))
    if v not in obitos_semana.keys():
        obitos_semana = obitos_semana.append(pd.Series({v: 0}))
casos_confirmados_semana = pd.Series({k: v for k, v in sorted(casos_confirmados_semana.items(), key=lambda item: item[0])})
obitos_semana = pd.Series({k: v for k, v in sorted(obitos_semana.items(), key=lambda item: item[0])})

In [ ]:
plt.rcParams.update({'font.size': 26})
plt.rc('text', usetex=True)

fig = plt.figure(figsize=(15, 7))
fig.tight_layout(pad=3.0)

plt.title('Fortaleza')
plt.xticks(rotation=45)

ax = fig.add_subplot(1, 1, 1)
res1, = ax.plot(casos_confirmados_semana, 'rs-', linewidth=3, label="Casos Confirmados")
ax.set_ylabel('Casos Confirmados', color='black')
ax.set_xlabel('Semana')

ax2 = ax.twinx()
res2, = ax2.plot(obitos_semana,'go-', linewidth=3, label='Óbitos')
ax2.set_ylabel('Óbitos', color='green')
ax2.tick_params(axis='y', labelcolor='green')

plt.legend(handles=[res1, res2])
plt.show()

In [ ]:
# Calculando mortes acumuladas por semana
obitos_acumulado_semana = obitos.groupby('SEMANA').size().cumsum()

# Calculando casos acumulados por semana
casos_confirmados_semana = casos_confirmados.groupby('SEMANA').size().cumsum()

# Eliminando as semanas que não tiveram mais que 50 óbitos
obitos_acumulado_semana = obitos_acumulado_semana[obitos_acumulado_semana > 50]

# Eliminando as semanas que não tiveram mais que 50 óbitos
## Obitos e Casos vao ter as mesmas semanas
casos_confirmados_semana = casos_confirmados_semana[casos_confirmados_semana.index.isin(obitos_acumulado_semana.index)]

### Gráfico Óbitos/Casos Acumulados por Semana (Fortaleza)

In [ ]:
plt.rcParams.update({'font.size': 26})

fig = plt.figure(figsize=(15, 7))
fig.tight_layout(pad=3.0)

plt.title('Fortaleza por Semana (Acumulado)')
plt.xticks(rotation=45)

ax = fig.add_subplot(1, 1, 1)
res1, = ax.plot(casos_confirmados_semana, 'rs-', linewidth=3, label="Casos Confirmados")
ax.set_ylabel('Casos Confirmados', color='black')
ax.set_xlabel('Semana')

ax2 = ax.twinx()
res2, = ax2.plot(obitos_acumulado_semana,'go-', linewidth=3, label='Óbitos')
ax2.set_ylabel('Óbitos', color='green')
ax2.tick_params(axis='y', labelcolor='green')

plt.legend(handles=[res1, res2])
plt.show()

---

## Encontrando Casos de Subnotificação

### Fórmulas do Cálculo dos Casos Ajustados
[Paper Referência](https://www.medrxiv.org/content/10.1101/2020.03.14.20036178v2.article-metrics)

#### Fator de Variabilidade
<br>
$\LARGE V_{FC} = \frac{\sum_{i=0}^{N} f_{F}r_i}{\sum_{i=0}^{n} f_{C}  r_i}$

$f_{F}$ é a fração da população com idade ${i}$ em Fortaleza.

$f_{C}$ é a fração da população com idade ${i}$ na Coréia do Sul.

${r_i}$ é taxa de mortalidade da idade ${i}$ na Coréia do Sul.

#### Casos Estimados
<br>
$\LARGE EC(T,t) = \frac{PD(t+d)}{V_{FC} x CFR_C}$

$PD(t+d)$ é o número de óbitos acumulados estimados no dia $t$ com um atraso de $d$ dias.

$d$ utilizado será <b>10</b>.

$V_{FC}$ é a taxa de variabilidade.

$CFR_C$ é a taxa de casos fatais na Coréia do Sul, que será <b>0.0213</b>.

---

### Faixa Etária de Fortaleza (2019)

| 0-9 | 10-19 | 20-39 | 40-59 | 60-69 | 70-79 | 80+ |
| --- | --- | --- | --- | --- | --- | --- |
| 377080 | 470620 | 965756 | 597194 | 141690 | 76771 | 40231 |

[Fonte](https://simda.sms.fortaleza.ce.gov.br/simda/populacao/faixa?faixaEtaria=2&modo=regional&ano_pop=2019)

### Faixa Etária da Coréia do Sul (2020)

| 0-9 | 10-19 | 20-39 | 40-59 | 60-69 | 70-79 | 80+ |
| --- | --- | --- | --- | --- | --- | --- |
| 4153813 | 4753258 | 13796133 | 16695543 | 6453706 | 3560646 | 1856084 |

[Fonte](https://www.populationpyramid.net/republic-of-korea/2020/)

### Taxa de Mortalidade (CFR) por Faixa Etária (Coréia do Sul) 
[Fonte](https://en.wikipedia.org/wiki/COVID-19_pandemic_in_South_Korea)

In [ ]:
taxa_mortalidade = np.array([0, 0, .12, .79, 2.31, 9.50, 24.91])

In [ ]:
fortaleza_faixa_etaria = np.array([377080, 470620, 965756, 597194, 141690, 76771, 40231])
coreia_sul_faixa_etaria = np.array([4153813, 4753258, 13796133, 16695543, 6453706, 3560646, 1856084])

In [ ]:
Vfc =  ((fortaleza_faixa_etaria / fortaleza_faixa_etaria.sum()) * taxa_mortalidade).sum() / ((coreia_sul_faixa_etaria / coreia_sul_faixa_etaria.sum()) * taxa_mortalidade).sum() 
Vfc

Com o Vfc < 1, então a população de Fortaleza é muito mais nova em comparação à Coréia do Sul. Vfc < 1 deve levar o CFR de Fortaleza a um número menor.

In [ ]:
DELAY = 10
CFR_COREIA = 0.0213

In [ ]:
# Última data que será utilizada para estimar os casos "reais". 
# Será 10 dias antes do último óbito
ultima_data = obitos['DATAOBITO'].sort_values(ascending=False).iloc[0] - pd.to_timedelta(DELAY, unit='d')

In [ ]:
# Casos tirando as semanas que não completeram os 50 óbitos iniciais
casos = casos_confirmados[casos_confirmados['SEMANA'].isin(obitos_acumulado_semana.index)]

# Casos até 10 dias antes do último óbito
casos = casos[(casos['DATANOTIFICACAO']) < ultima_data - pd.to_timedelta(DELAY, unit='d')]

In [ ]:
EC_dict = dict()
for day in casos['DATANOTIFICACAO'].sort_values(ascending=True).unique():
    dd = day + pd.to_timedelta(DELAY, unit='d')
    EC = obitos[obitos['DATAOBITO'] == dd]['DATAOBITO'].count() / (Vfc * CFR_COREIA)
    EC_dict[str(dd.strftime('%Y-%m-%d'))] = int(EC)

In [ ]:
# Novo DataFrame com os casos estimados
EC_df = pd.DataFrame.from_dict(EC_dict.items())
EC_df.columns = ['DATA', 'TOTAL']

EC_df['DATA'] = pd.to_datetime(EC_df['DATA'])
EC_df['SEMANA'] = EC_df['DATA'].dt.strftime('%W')

In [ ]:
# Acumulativo por Semana
EC_semana = EC_df.groupby('SEMANA')[['SEMANA', 'TOTAL']].sum().cumsum()

# Acumulativo por dia
EC_dia = EC_df.groupby('DATA')[['DATA', 'TOTAL']].sum().cumsum()

# Casos confirmados semana
casos_confirmados_semana = casos_confirmados.groupby('SEMANA').size().cumsum()

# Somente com as mesmas semanas
casos_confirmados_semana = casos_confirmados_semana[casos_confirmados_semana.index.isin(EC_df['SEMANA'].unique())]

# Casos confirmados por dia
casos_confirmados_dia = casos_confirmados.groupby('DATANOTIFICACAO').size().cumsum()
# Somente com os mesmos dias
casos_confirmados_dia = casos_confirmados_dia[casos_confirmados_dia.index.isin(EC_df['DATA'].unique())]

### Novo Gráfico dos Casos Corrigidos

#### Por Semana

In [ ]:
plt.rcParams.update({'font.size': 26})

fig = plt.figure(figsize=(15, 7))
fig.tight_layout(pad=3.0)

plt.title('Casos em Fortaleza por Semana (Acumulado)')
plt.xticks(rotation=45)

ax = fig.add_subplot(1, 1, 1)
res1, = ax.plot(casos_confirmados_semana, 'rs-', linewidth=3, label="Casos Confirmados")
res2, = ax.plot(EC_semana,'go-', linewidth=3, label='Casos Corrigidos')

ax.set_xlabel('Semana', color='black')
ax.set_ylabel('Casos', color='black')

plt.legend(handles=[res1, res2])
plt.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=EC_semana.index, y=EC_semana['TOTAL'], name='Casos Corrigidos',
                         hoverinfo='y',
                         line=dict(color='green', width=3)))

fig.add_trace(go.Scatter(x=casos_confirmados_semana.index, y=casos_confirmados_semana, name='Casos Confirmados',
                         hoverinfo='y',
                         line=dict(color='red', width=3)))

fig.update_yaxes(tickvals=list(range(0, int(max(EC_semana['TOTAL']) * 1.2), 30000)))
fig.update_xaxes(tickvals=casos_confirmados_semana.index)
fig.update_layout(hovermode="x unified")

fig.update_layout(title='Casos em Fortaleza por Semana (Acumulado)',
                   xaxis_title='Semana',
                   yaxis_title='Casos (Milhares)',
                   width=600, height=500)

#### Fator Mutiplicador Semana

In [ ]:
for semana, valor in zip(EC_semana.index, EC_semana['TOTAL'].values / casos_confirmados_semana.values * 100):
    print(f'Semana {semana} teve {round(round(valor, 2) - 100, 2)}% de casos não notificados.')

#### Por Dia

In [ ]:
plt.rcParams.update({'font.size': 26})

fig = plt.figure(figsize=(21, 7))
fig.tight_layout(pad=3.0)

plt.title('Casos em Fortaleza por Dia (Acumulado)')
plt.xticks(rotation=45)

ax = fig.add_subplot(1, 1, 1)
res1, = ax.plot(casos_confirmados_dia, 'rs-', linewidth=2, label="Casos Confirmados")
res2, = ax.plot(EC_dia,'go-', linewidth=2, label='Casos Corrigidos')

ax.set_xlabel('Data', color='black')
ax.set_ylabel('Casos', color='black')

plt.legend(handles=[res1, res2])
plt.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=EC_dia.index, y=EC_dia['TOTAL'], name='Casos Corrigidos',
                         hoverinfo='y',
                         line=dict(color='green', width=3)))

fig.add_trace(go.Scatter(x=casos_confirmados_dia.index, y=casos_confirmados_dia, name='Casos Confirmados',
                         hoverinfo='y',
                         line=dict(color='red', width=3)))

fig.update_yaxes(tickvals=list(range(0, int(max(EC_dia['TOTAL']) * 1.2), 30000)))
# fig.update_xaxes(tickvals=casos_confirmados_dia.index)
fig.update_layout(hovermode="x unified")


fig.update_layout(title='Casos em Fortaleza por Dia (Acumulado)',
                   xaxis_title='Dias',
                   yaxis_title='Casos (Milhares)',
                   width=600, height=500)